In [ ]:
#
# A notebook to unreconstruct paleotopographies from previously published work
#

import pygplates
import sys
import glob
import numpy as np
import matplotlib.pyplot as plt
sys.path.append('../../passive_margins')
sys.path.append('../../atom_utils')
sys.path.append('../../GPlatesClassStruggle')

from pigplates import sphere_tools as pigsph
from reconstruct_atom_rasters import *


%matplotlib inline
%load_ext autoreload
%autoreload 2

from raster_reconstruction_classes import *
from reconstruction_classes import *

rotation_filename = '/Users/Simon/Data/GPlates/PublishedModels/Scotese_PaleoAtlas/PALEOMAP_Global_Plate_Model/PALEOMAP_PlateModel.rot'
static_polygon_filename = '/Users/Simon/Data/GPlates/PublishedModels/Scotese_PaleoAtlas/PALEOMAP_Global_Plate_Model/PALEOMAP_PlatePolygons.gpml'


PALEOMAP = ReconstructionModel('PALEOMAP')
PALEOMAP.add_rotation_model(rotation_filename)
PALEOMAP.add_static_polygons(static_polygon_filename)


# Loop over each image in the PALEOMAP data set
# First, get list of all image files in directory
file_list = glob.glob('/Users/Simon/Data/GPlates/PublishedModels/Scotese_PaleoAtlas/PaleoDEMS_netcdf/*.nc')

time_list = []
for file_name in file_list:
    time_list.append(file_name.split('_')[-2][:-2])



In [32]:
# This step is necessary because the PALEOMAP polygons include some 'invalid geometries' (e.g. polylines, and polygons with 2 points)
# So we need to clean these up, and ignore oceanic polygons that are only valid at present day
sp = []
youngest_polygon_to_take = 100.

for polygon in pygplates.FeatureCollection(static_polygon_filename):
    for geometry in polygon.get_all_geometries():
        if type(geometry)!=pygplates.PolylineOnSphere:
            if polygon.get_valid_time()[0]>youngest_polygon_to_take:
                sp.append(polygon)


for filename,reconstruction_time in zip(file_list,time_list):
    
    print reconstruction_time
    
    ptopo = GplatesRaster(filename)

    from_time = np.float(reconstruction_time)
    to_time = 0.

    (unreconstructed_point_lons,
     unreconstructed_point_lats,
     topography_points) = reconstruct_raster(ptopo, sp, PALEOMAP.rotation_model,
                                             from_time, to_time, 
                                             grid_sampling=2.0)

    plt.figure(figsize=(20,9))
    plt.scatter(unreconstructed_point_lons,unreconstructed_point_lats,
                c=topography_points,s=12,vmin=-3000,vmax=3000)
    plt.colorbar()
    plt.savefig('./images/UnreconstructedPaleoDEM_%sMa.png' % reconstruction_time)
    #plt.show()
    plt.close()


217.8
Reconstruct static polygons...
Find static polygons...
Group by polygons...
Reverse reconstruct points...
Sample present-day grid...
455.7
Reconstruct static polygons...
Find static polygons...
Group by polygons...
Reverse reconstruct points...
Sample present-day grid...
354
Reconstruct static polygons...
Find static polygons...
Group by polygons...
Reverse reconstruct points...
Sample present-day grid...
465
Reconstruct static polygons...
Find static polygons...
Group by polygons...
Reverse reconstruct points...
Sample present-day grid...
115.8
Reconstruct static polygons...
Find static polygons...
Group by polygons...
Reverse reconstruct points...
Sample present-day grid...
14.9
Reconstruct static polygons...
Find static polygons...
Group by polygons...
Reverse reconstruct points...
Sample present-day grid...
460
Reconstruct static polygons...
Find static polygons...
Group by polygons...
Reverse reconstruct points...
Sample present-day grid...
60.6
Reconstruct static polygons..